### Exploratory Data Analysis with Python

We will explore the NYC MTA turnstile data set. These data files are from the New York Subway. It tracks the hourly entries and exits to turnstiles (UNIT) by day in the subway system.

Here is an [example of what you could do with the data](https://jameskao.me/analyzing-the-nyc-subway-dataset/). James Kao investigates how subway ridership is affected by incidence of rain.

## Exercise 1

- Download at least 2 weeks worth of [MTA turnstile data](http://web.mta.info/developers/turnstile.html) (You can do this manually or via Python)
- Open up a file, use csv reader to read it, make a python dict where there is a key for each (C/A, UNIT, SCP, STATION). These are the first four columns. The value for this key should be a list of lists. Each list in the list is the rest of the columns in a row. For example, one key-value pair should look like


        {    ('A002','R051','02-00-00','LEXINGTON AVE'):    
             [
               ['NQR456', 'BMT', '01/03/2015', '03:00:00', 'REGULAR', '0004945474', '0001675324'],          
                 ['NQR456', 'BMT', '01/03/2015', '07:00:00', 'REGULAR', '0004945478', '0001675333'],  
                ['NQR456', 'BMT', '01/03/2015', '11:00:00', 'REGULAR', '0004945515', '0001675364'],
              ...   
         ] 
        }

*Store all the weeks in a data structure of your choosing*

In [6]:
# information about the wget module here: https://pypi.python.org/pypi/wget

import wget

for date in ['170527', '170520', '170513', '170506']:
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'.format(date)
    wget.download(url)
    print(date, 'file downloaded')
    

170527 file downloaded
170520 file downloaded
170513 file downloaded
170506 file downloaded


In [25]:
import csv, glob # glob allows you to join files
from collections import defaultdict # lets you set a default dict value type - i.e. list

def read_csv(file):
    '''makes a dictionary of the mta data where, for each row, the key is the first four columns of data and the values
    are the rest of the columns'''
    turnstile_reading = defaultdict(list) # set the default value to list
    
    with open(file, 'r') as csv_file:
        mta_reader = csv.reader(csv_file)
        
        for i, row in enumerate(mta_reader): # returns a tuple with numbered items 
            if i == 0: # passes the header row (column names)
                continue
                
            turnstile_info = tuple(row[:4]) # grabs the first four items in the row
            count_reading = row[4:] # grabs items 5:end of the row
            turnstile_reading[turnstile_info].append(count_reading) # make a key value pair in the turnstile reading dict
            # where each key is the turnstile information and each value is the turnstile count
            
    return turnstile_reading

In [26]:
weekly_data_dicts = [read_csv(csvfile) for csvfile in glob.glob('turnstile_*.txt')] 
# returns all files that match the filename 'turnstile_*.txt' - the wildcard (*) is for the 
# dates which are different for each file. glob.glob returns each file 
# weekly_data_dicts is a list that contains dictionaries - one dictionary for each week of data
# in each dict, the keys are turnstile information and the values are the information for that 
# time period 

In [37]:
print(len(weekly_data_dicts))

4


In [27]:
sample_dict = list(weekly_data_dicts[0].items())[:2] # makes a list called sample_dict with the 
# which takes the first two items from the first dict in weekly_data_dicts
print(sample_dict)


[(('A002', 'R051', '02-00-00', '59 ST'), [['NQR456W', 'BMT', '04/29/2017', '00:00:00', 'REGULAR', '0006157740', '0002085315                                            '], ['NQR456W', 'BMT', '04/29/2017', '04:00:00', 'REGULAR', '0006157777', '0002085319                                            '], ['NQR456W', 'BMT', '04/29/2017', '08:00:00', 'REGULAR', '0006157810', '0002085353                                            '], ['NQR456W', 'BMT', '04/29/2017', '12:00:00', 'REGULAR', '0006157963', '0002085453                                            '], ['NQR456W', 'BMT', '04/29/2017', '16:00:00', 'REGULAR', '0006158212', '0002085529                                            '], ['NQR456W', 'BMT', '04/29/2017', '20:00:00', 'REGULAR', '0006158521', '0002085589                                            '], ['NQR456W', 'BMT', '04/30/2017', '00:00:00', 'REGULAR', '0006158637', '0002085612                                            '], ['NQR456W', 'BMT', '04/30/2017', '04:00:00', 'REGULAR',

In [28]:
from pprint import pprint

pprint(sample_dict)

[(('A002', 'R051', '02-00-00', '59 ST'),
  [['NQR456W',
    'BMT',
    '04/29/2017',
    '00:00:00',
    'REGULAR',
    '0006157740',
    '0002085315                                            '],
   ['NQR456W',
    'BMT',
    '04/29/2017',
    '04:00:00',
    'REGULAR',
    '0006157777',
    '0002085319                                            '],
   ['NQR456W',
    'BMT',
    '04/29/2017',
    '08:00:00',
    'REGULAR',
    '0006157810',
    '0002085353                                            '],
   ['NQR456W',
    'BMT',
    '04/29/2017',
    '12:00:00',
    'REGULAR',
    '0006157963',
    '0002085453                                            '],
   ['NQR456W',
    'BMT',
    '04/29/2017',
    '16:00:00',
    'REGULAR',
    '0006158212',
    '0002085529                                            '],
   ['NQR456W',
    'BMT',
    '04/29/2017',
    '20:00:00',
    'REGULAR',
    '0006158521',
    '0002085589                                            '],
   ['NQR456W',
    'BMT

## Exercise 2
- Let's turn this into a time series.

 For each key (basically the control area, unit, device address and station of a specific turnstile), have a list again, but let the list be comprised of just the point in time and the cumulative count of entries.

This basically means keeping only the date, time, and entries fields in each list. You can convert the date and time into datetime objects -- That is a python class that represents a point in time. You can combine the date and time fields into a string and use the [dateutil](https://dateutil.readthedocs.io/en/stable/) module to convert it into a datetime object.

Your new dict should look something like
 
    {    ('A002','R051','02-00-00','LEXINGTON AVE'):    
             [
                [datetime.datetime(2013, 3, 2, 3, 0), 3788],
                [datetime.datetime(2013, 3, 2, 7, 0), 2585],
                [datetime.datetime(2013, 3, 2, 12, 0), 10653],
                [datetime.datetime(2013, 3, 2, 17, 0), 11016],
                [datetime.datetime(2013, 3, 2, 23, 0), 10666],
                [datetime.datetime(2013, 3, 3, 3, 0), 10814],
                [datetime.datetime(2013, 3, 3, 7, 0), 10229],
                ...
              ],
     ....
     }



In [31]:
from datetime import datetime
from dateutil.parser import parse

def convert_to_time_series(week_data_dict):
    '''take a dictionary of information about each turnstile and returns a new dict  
    containing the turnstile (key) and the timestamp and cumulative entries as the values'''
    time_series = defaultdict(list) # sets the default dict to list - creates an empty list for each key initialized
    
    for i, (turnstile, row_data) in enumerate(week_data_dict.items()):
        if i % 100 == 0:
            print('Processing turnstile: ', turnstile)
        for linename, division, date, time, desc, cum_entries, cum_exits in row_data:
            timestamp = parse('%sT%s' %(date, time))
            time_series[turnstile].append([timestamp, int(cum_entries)])
            
    return time_series

In [32]:
weekly_time_series = list(map(convert_to_time_series, weekly_data_dicts)) # map applies the 
# specified function (convert_to_time_series) to each item in a list (weekly_data_dicts)

Processing turnstile:  ('A002', 'R051', '02-00-00', '59 ST')
Processing turnstile:  ('A030', 'R083', '01-06-00', '23 ST')
Processing turnstile:  ('A049', 'R088', '02-01-01', 'CORTLANDT ST')
Processing turnstile:  ('A084', 'R125', '01-00-03', 'BROAD ST')
Processing turnstile:  ('B024', 'R211', '00-00-03', 'KINGS HWY')
Processing turnstile:  ('C012', 'R258', '01-03-01', '4AV-9 ST')
Processing turnstile:  ('E001', 'R368', '00-00-03', '9 AV')
Processing turnstile:  ('H007', 'R248', '00-03-01', '1 AV')
Processing turnstile:  ('H040', 'R376', '00-00-01', 'EAST 105 ST')
Processing turnstile:  ('J030', 'R005', '00-00-03', '85 ST-FOREST PK')
Processing turnstile:  ('N013', 'R035', '02-00-04', '168 ST')
Processing turnstile:  ('N049', 'R084', '01-02-02', '59 ST COLUMBUS')
Processing turnstile:  ('N067', 'R012', '00-05-01', '34 ST-PENN STA')
Processing turnstile:  ('N089', 'R139', '00-03-01', 'CANAL ST')
Processing turnstile:  ('N102', 'R127', '01-05-01', 'JAY ST-METROTEC')
Processing turnstile: 

In [39]:
sample = list(weekly_time_series[0].items())[0:5]
pprint(sample)

[(('A002', 'R051', '02-00-00', '59 ST'),
  [[datetime.datetime(2017, 4, 29, 0, 0), 6157740],
   [datetime.datetime(2017, 4, 29, 4, 0), 6157777],
   [datetime.datetime(2017, 4, 29, 8, 0), 6157810],
   [datetime.datetime(2017, 4, 29, 12, 0), 6157963],
   [datetime.datetime(2017, 4, 29, 16, 0), 6158212],
   [datetime.datetime(2017, 4, 29, 20, 0), 6158521],
   [datetime.datetime(2017, 4, 30, 0, 0), 6158637],
   [datetime.datetime(2017, 4, 30, 4, 0), 6158654],
   [datetime.datetime(2017, 4, 30, 8, 0), 6158661],
   [datetime.datetime(2017, 4, 30, 12, 0), 6158703],
   [datetime.datetime(2017, 4, 30, 16, 0), 6158797],
   [datetime.datetime(2017, 4, 30, 20, 0), 6159015],
   [datetime.datetime(2017, 5, 1, 0, 0), 6159107],
   [datetime.datetime(2017, 5, 1, 4, 0), 6159116],
   [datetime.datetime(2017, 5, 1, 8, 0), 6159166],
   [datetime.datetime(2017, 5, 1, 12, 0), 6159331],
   [datetime.datetime(2017, 5, 1, 16, 0), 6159595],
   [datetime.datetime(2017, 5, 1, 20, 0), 6160374],
   [datetime.datetim

## Exercise 3
- These counts are cumulative every n hours. We want total daily entries. 

Now make it that we again have the same keys, but now we have a single value for a single day, which is not cumulative counts but the total number of passengers that entered through this turnstile on this day.


In [49]:
def combine_weeks(weekly_dicts):
    combined = defaultdict(list)
    
    for week in weekly_dicts:
        for turnstile, time_series in week.items():
            combined[turnstile] += time_series
            
    return combined

In [51]:
combined_time_series = combine_weeks(weekly_time_series)

In [53]:
list(combined_time_series.items())[:5]

[(('A002', 'R051', '02-00-00', '59 ST'),
  [[datetime.datetime(2017, 4, 29, 0, 0), 6157740],
   [datetime.datetime(2017, 4, 29, 4, 0), 6157777],
   [datetime.datetime(2017, 4, 29, 8, 0), 6157810],
   [datetime.datetime(2017, 4, 29, 12, 0), 6157963],
   [datetime.datetime(2017, 4, 29, 16, 0), 6158212],
   [datetime.datetime(2017, 4, 29, 20, 0), 6158521],
   [datetime.datetime(2017, 4, 30, 0, 0), 6158637],
   [datetime.datetime(2017, 4, 30, 4, 0), 6158654],
   [datetime.datetime(2017, 4, 30, 8, 0), 6158661],
   [datetime.datetime(2017, 4, 30, 12, 0), 6158703],
   [datetime.datetime(2017, 4, 30, 16, 0), 6158797],
   [datetime.datetime(2017, 4, 30, 20, 0), 6159015],
   [datetime.datetime(2017, 5, 1, 0, 0), 6159107],
   [datetime.datetime(2017, 5, 1, 4, 0), 6159116],
   [datetime.datetime(2017, 5, 1, 8, 0), 6159166],
   [datetime.datetime(2017, 5, 1, 12, 0), 6159331],
   [datetime.datetime(2017, 5, 1, 16, 0), 6159595],
   [datetime.datetime(2017, 5, 1, 20, 0), 6160374],
   [datetime.datetim

In [54]:
def convert_to_daily_time_series(combined_time_series):
    
    '''Iterates over the combined_time_series, calling the daily_calculation function to 
    calculate total entries in each day for each turnstile. Returns the total entries for each 
    turnstile in combined_time_series'''
    
    turnstile_daily_time_series = {}
    
    for i, (turnstile, time_series) in enumerate(combined_time_series.items()):
        print('Processing turnstile', turnstile)
        turnstile_daily_time_series[turnstile] = daily_calculation(time_series) # Creates a dict
        # where the key is the turnstile and the value is the total number of entries
    
    return turnstile_daily_time_series 

In [61]:
from itertools import groupby

def count_within_normal_bounds(count):
    '''Checks whether the calculated turnstile count is within a normal range (i.e. greater
    than 0 and less than 10,000). If count is none, returns true'''
    
    if count is None:
        return True
    else:
        return 10000 > count >= 0
    
def daily_calculation(time_series):
    '''Calculates the entry count for a each turnstile each day'''
    daily_time_series = []
    
    def day_of_timestamp(entry):
        '''takes an entry from from the time series and returns the date'''
        
        timestamp, tot_entries = entry
        return timestamp.date()
    
    count_on_previous_day = None
    for day, entries_on_this_day in groupby(time_series, key=day_of_timestamp):
        cum_entry_count_on_day = max([count for time, count in entries_on_this_day])
        
        if count_on_previous_day is None:
            daily_entries = None
        else:
            daily_entries = cum_entry_count_on_day - count_on_previous_day
        
        count_on_previous_day = cum_entry_count_on_day
        
        if count_within_normal_bounds(daily_entries):
            daily_time_series.append((day, daily_entries))
            
        else: 
            print('Warning: Abnormal entry count found on day {}: {}'.format(day, daily_entries))
            daily_time_series.append((day, None))

    return daily_time_series

In [62]:
daily_time_series = convert_to_daily_time_series(combined_time_series)

Processing turnstile ('A002', 'R051', '02-00-00', '59 ST')
Processing turnstile ('A002', 'R051', '02-00-01', '59 ST')
Processing turnstile ('A002', 'R051', '02-03-00', '59 ST')
Processing turnstile ('A002', 'R051', '02-03-01', '59 ST')
Processing turnstile ('A002', 'R051', '02-03-02', '59 ST')
Processing turnstile ('A002', 'R051', '02-03-03', '59 ST')
Processing turnstile ('A002', 'R051', '02-03-04', '59 ST')
Processing turnstile ('A002', 'R051', '02-03-05', '59 ST')
Processing turnstile ('A002', 'R051', '02-03-06', '59 ST')
Processing turnstile ('A002', 'R051', '02-05-00', '59 ST')
Processing turnstile ('A002', 'R051', '02-05-01', '59 ST')
Processing turnstile ('A002', 'R051', '02-06-00', '59 ST')
Processing turnstile ('A006', 'R079', '00-00-00', '5 AV/59 ST')
Processing turnstile ('A006', 'R079', '00-00-01', '5 AV/59 ST')
Processing turnstile ('A006', 'R079', '00-00-02', '5 AV/59 ST')
Processing turnstile ('A006', 'R079', '00-00-03', '5 AV/59 ST')
Processing turnstile ('A006', 'R079'

In [63]:
pprint(daily_time_series[('A006', 'R079', '00-00-03', '5 AV/59 ST')])

[(datetime.date(2017, 4, 29), None),
 (datetime.date(2017, 4, 30), 409),
 (datetime.date(2017, 5, 1), 1021),
 (datetime.date(2017, 5, 2), 1071),
 (datetime.date(2017, 5, 3), 1094),
 (datetime.date(2017, 5, 4), 1097),
 (datetime.date(2017, 5, 5), 840),
 (datetime.date(2017, 5, 6), 527),
 (datetime.date(2017, 5, 7), 384),
 (datetime.date(2017, 5, 8), 975),
 (datetime.date(2017, 5, 9), 1060),
 (datetime.date(2017, 5, 10), 1092),
 (datetime.date(2017, 5, 11), 1090),
 (datetime.date(2017, 5, 12), 973),
 (datetime.date(2017, 5, 13), 411),
 (datetime.date(2017, 5, 14), 520),
 (datetime.date(2017, 5, 15), 1015),
 (datetime.date(2017, 5, 16), 1124),
 (datetime.date(2017, 5, 17), 1160),
 (datetime.date(2017, 5, 18), 1121),
 (datetime.date(2017, 5, 19), 1128),
 (datetime.date(2017, 5, 20), 539),
 (datetime.date(2017, 5, 21), 575),
 (datetime.date(2017, 5, 22), 939),
 (datetime.date(2017, 5, 23), 1025),
 (datetime.date(2017, 5, 24), 1076),
 (datetime.date(2017, 5, 25), 1055),
 (datetime.date(2017,

## Exercise 4
- We will plot the daily time series for a turnstile.

In ipython notebook, add this to the beginning of your next cell:    

    %matplotlib inline

This will make your matplotlib graphs integrate nicely with the notebook.
To plot the time series, import matplotlib with 

    import matplotlib.pyplot as plt

Take the list of [(date1, count1), (date2, count2), ...], for the turnstile and turn it into two lists:
dates and counts. This should plot it:

    plt.figure(figsize=(10,3))
    plt.plot(dates,counts)


## Exercise 5
- So far we've been operating on a single turnstile level, let's combine turnstiles in the same ControlArea/Unit/Station combo. There are some ControlArea/Unit/Station groups that have a single turnstile, but most have multiple turnstilea-- same value for the C/A, UNIT and STATION columns, different values for the SCP column.

We want to combine the numbers together -- for each ControlArea/UNIT/STATION combo, for each day, add the counts from each turnstile belonging to that combo.


## Exercise 6
- Similarly, combine everything in each station, and come up with a time series of `[(date1, count1),(date2,count2),...]` type of time series for each STATION, by adding up all the turnstiles in a station.

## Exercise 7
- Plot the time series for a station

## Exercise 8
- Make one list of counts for **one** week for one station. Monday's count, Tuesday's count, etc. so it's a list of 7 counts.
Make the same list for another week, and another week, and another week.
`plt.plot(week_count_list)` for every `week_count_list` you created this way. You should get a rainbow plot of weekly commute numbers on top of each other.



## Exercise 9
- Over multiple weeks, sum total ridership for each station and sort them, so you can find out the stations with the highest traffic during the time you investigate

## Exercise 10
- Make a single list of these total ridership values and plot it with `plt.hist(total_ridership_counts)` to get an idea about the distribution of total ridership among different stations.   
This should show you that most stations have a small traffic, and the histogram bins for large traffic volumes have small bars.

*Additional Hint*:    
If you want to see which stations take the meat of the traffic, you can sort the total ridership counts and make a `plt.bar` graph. For this, you want to have two lists: the indices of each bar, and the values. The indices can just be `0,1,2,3,...`, so you can do 

    indices = range(len(total_ridership_values))
    plt.bar(indices, total_ridership_values)

    